In [ ]:
# Install package if havent
!pip install ipython-autotime
!pip install transformers
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.4 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=7d79aef77de7002ff706e88fd5f9415298d30ca7aab4df551ffdd3a2c74ac844
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-tr

<h1> 3. Feature Engineering

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

# Load data from colab
from google.colab import drive
drive.mount('/content/drive')

# For feature engineering
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.sentiment import SentimentIntensityAnalyzer
from transformers import BertTokenizer, BertModel
import torch
from sentence_transformers import SentenceTransformer

import warnings
warnings.filterwarnings('ignore')
%load_ext autotime

Mounted at /content/drive
time: 296 µs (started: 2023-10-24 14:33:27 +00:00)


In [ ]:
df = pd.read_csv('/content/drive/My Drive/BT4012 Group 06/Data/feature_engineering_data_v3.csv')
df.head()

In [ ]:
df.columns

Index(['job_id', 'title', 'location', 'department', 'salary_range',
       'company_profile', 'description', 'requirements', 'benefits',
       'telecommuting', 'has_company_logo', 'has_questions', 'employment_type',
       'required_experience', 'required_education', 'industry', 'function',
       'fraudulent', 'min_salary', 'max_salary', 'noise_removed_title',
       'noise_removed_company_profile', 'noise_removed_description',
       'noise_removed_requirements', 'noise_removed_benefits',
       'tokenized_title', 'pos_tags_title', 'tokenized_company_profile',
       'pos_tags_company_profile', 'tokenized_description',
       'pos_tags_description', 'tokenized_requirements',
       'pos_tags_requirements', 'tokenized_benefits', 'pos_tags_benefits'],
      dtype='object')

time: 4.08 ms (started: 2023-10-11 03:46:18 +00:00)


<H3> Location Features

In [ ]:
df['country'] = ''
df['province'] = ''
df['city'] = ''
for i in range(len(df)):
  location = df['location'].iloc[i].split(',')
  df['country'].iloc[i] = location[0]
  if len(location) > 1:
    df['province'].iloc[i] = location[1]
  if len(location) > 2:
    df['city'].iloc[i] = location[2]

# drop the original column
df = df.drop(['location'], axis = 1)

time: 19.1 s (started: 2023-10-17 07:36:52 +00:00)


<h3> Word Count Features

In [ ]:
df['length of profile name'] = df['company_profile'].apply(lambda x: len(x))

time: 13.9 ms (started: 2023-10-11 08:51:53 +00:00)


In [ ]:
df['length of description'] = df['description'].apply(lambda x: len(x))

time: 27 ms (started: 2023-10-11 08:51:54 +00:00)


In [ ]:
df['length of requirements'] = df['requirements'].apply(lambda x: len(x))

time: 21.5 ms (started: 2023-10-11 08:51:55 +00:00)


In [ ]:
df['length of benefits'] = df['benefits'].apply(lambda x: len(x))

time: 23.5 ms (started: 2023-10-11 08:51:56 +00:00)


<h3> Capital Letter Features

In [ ]:
df['capital_words_number_profile'] = df['company_profile'].apply(lambda x: sum(map(str.isupper,x.split())))

time: 379 ms (started: 2023-10-11 08:51:57 +00:00)


In [ ]:
df['capital_words_number_description'] = df['description'].apply(lambda x: sum(map(str.isupper,x.split())))

time: 750 ms (started: 2023-10-11 08:51:57 +00:00)


In [ ]:
df['capital_words_number_requirements'] = df['requirements'].apply(lambda x: sum(map(str.isupper,x.split())))

time: 251 ms (started: 2023-10-11 08:51:58 +00:00)


In [ ]:
df['capital_words_number_benefits'] = df['benefits'].apply(lambda x: sum(map(str.isupper,x.split())))

time: 96.6 ms (started: 2023-10-11 08:51:59 +00:00)


<h3> Sentence Count Features

In [ ]:
df['number of sentences_profile'] = df['company_profile'].apply(lambda x: len(nltk.sent_tokenize(x)))

time: 1.89 s (started: 2023-10-11 08:52:00 +00:00)


In [ ]:
df['number of sentences_description'] = df['description'].apply(lambda x: len(nltk.sent_tokenize(x)))

time: 2.66 s (started: 2023-10-11 08:52:02 +00:00)


In [ ]:
df['number of sentences_requirements'] = df['requirements'].apply(lambda x: len(nltk.sent_tokenize(x)))

time: 1.09 s (started: 2023-10-11 08:52:04 +00:00)


In [ ]:
df['number of sentences_benefits'] = df['benefits'].apply(lambda x: len(nltk.sent_tokenize(x)))

time: 722 ms (started: 2023-10-11 08:52:05 +00:00)


<h3>Count of Stop Words

In [ ]:
def count_stopwords(text):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)
    stopwords_x = [w for w in word_tokens if w in stop_words]
    return len(stopwords_x)

df['number of stopwords_profile'] = df['description'].apply(count_stopwords)

time: 30.8 s (started: 2023-10-11 08:52:06 +00:00)


In [ ]:
df = df.rename(columns={'number of stopwords_profile': 'number of stopwords_profile_description'})

time: 15.7 ms (started: 2023-10-11 08:52:37 +00:00)


In [ ]:
df['number of stopwords_profile'] = df['company_profile'].apply(count_stopwords)

time: 20.4 s (started: 2023-10-11 08:52:37 +00:00)


In [ ]:
df['number of stopwords_requirements'] = df['requirements'].apply(count_stopwords)

time: 22.5 s (started: 2023-10-11 08:52:57 +00:00)


In [ ]:
df['number of stopwords_benefit'] = df['benefits'].apply(count_stopwords)

time: 9.85 s (started: 2023-10-11 08:53:20 +00:00)


<h3> Sentiment Features

In [ ]:
sia = SentimentIntensityAnalyzer()
def get_sentiment_score(text):
    return sia.polarity_scores(text)["compound"]
df['sentiment score_profile'] = df['company_profile'].apply(get_sentiment_score)

time: 30.8 s (started: 2023-10-11 08:53:30 +00:00)


In [ ]:
df['sentiment score_requirement'] = df['requirements'].apply(get_sentiment_score)

time: 18.3 s (started: 2023-10-11 08:54:01 +00:00)


In [ ]:
df['sentiment score_benefits'] = df['benefits'].apply(get_sentiment_score)

time: 9.09 s (started: 2023-10-11 08:54:19 +00:00)


In [ ]:
df['sentiment score_description'] = df['description'].apply(get_sentiment_score)

time: 49.2 s (started: 2023-10-11 08:54:28 +00:00)


<h3> POS Tagging Features

In [ ]:
# NOUN
col_to_include = ['company_profile','description','requirements','benefits']
for col in col_to_include:
  df['num_of_nouns_' + col] = df['pos_tags_' + col].apply(lambda x:eval(x).count('NOUN'))
  df['nouns_vs_words_' + col] = df['num_of_nouns_' + col] / df['pos_tags_' + col].apply(lambda x:len(eval(x)))
  df['nouns_vs_length_' + col] = df['num_of_nouns_' + col] / df['pos_tags_' + col].apply(lambda x:sum(len(string) for string in eval(x)))

time: 14.5 s (started: 2023-10-18 07:15:41 +00:00)


In [ ]:
# VERB
for col in col_to_include:
  df['num_of_verbs_' + col] = df['pos_tags_' + col].apply(lambda x:eval(x).count('VERB'))
  df['verbs_vs_words_' + col] = df['num_of_verbs_' + col] / df['pos_tags_' + col].apply(lambda x:len(eval(x)))
  df['verbs_vs_length_' + col] = df['num_of_verbs_' + col] / df['pos_tags_' + col].apply(lambda x:sum(len(string) for string in eval(x)))

time: 14.5 s (started: 2023-10-18 07:15:56 +00:00)


In [ ]:
# ADJ
for col in col_to_include:
  df['num_of_adjs_' + col] = df['pos_tags_' + col].apply(lambda x:eval(x).count('ADJ'))
  df['adjs_vs_words_' + col] = df['num_of_adjs_' + col] / df['pos_tags_' + col].apply(lambda x:len(eval(x)))
  df['adjs_vs_length_' + col] = df['num_of_adjs_' + col] / df['pos_tags_' + col].apply(lambda x:sum(len(string) for string in eval(x)))

time: 14.5 s (started: 2023-10-18 07:16:10 +00:00)


In [ ]:
# PROPN
for col in col_to_include:
  df['num_of_propns_' + col] = df['pos_tags_' + col].apply(lambda x:eval(x).count('PROPN'))
  df['propns_vs_words_' + col] = df['num_of_propns_' + col] / df['pos_tags_' + col].apply(lambda x:len(eval(x)))
  df['propns_vs_length_' + col] = df['num_of_propns_' + col] / df['pos_tags_' + col].apply(lambda x:sum(len(string) for string in eval(x)))

time: 14.6 s (started: 2023-10-18 07:32:49 +00:00)


<h3> URL Feature

In [ ]:
col_to_include = ['company_profile','description','requirements','benefits']
for col in col_to_include:
    df['url_count'] = df[col].str.count('url') + df[col].str.count('URL')

time: 1.22 s (started: 2023-10-18 07:37:23 +00:00)


<h3>Embedding Features

In [ ]:
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

time: 7.88 s (started: 2023-10-17 09:25:30 +00:00)


In [ ]:
def get_embeddings(text):
  tokens = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
  with torch.no_grad():
    output = model(**tokens)
    embeddings = output.last_hidden_state
  return embeddings

time: 735 µs (started: 2023-10-17 09:21:39 +00:00)


In [ ]:
df['embeddings_profile'] = df['company_profile'].apply(get_embeddings)

time: 2h 31min 30s (started: 2023-10-12 15:57:08 +00:00)


In [ ]:
df['embeddings_profile']

0        [[[tensor(-0.2207), tensor(-0.1446), tensor(-0...
1        [[[tensor(-0.3885), tensor(-0.1363), tensor(-0...
2        [[[tensor(-0.0087), tensor(0.0359), tensor(-0....
3        [[[tensor(-0.2483), tensor(0.2116), tensor(0.1...
4        [[[tensor(-0.1210), tensor(-0.3567), tensor(-0...
                               ...                        
17875    [[[tensor(0.0148), tensor(-0.3094), tensor(-0....
17876    [[[tensor(0.1798), tensor(-0.1232), tensor(-0....
17877    [[[tensor(0.0816), tensor(-0.0776), tensor(-0....
17878    [[[tensor(-0.4404), tensor(0.1208), tensor(-0....
17879    [[[tensor(0.0148), tensor(-0.3094), tensor(-0....
Name: embeddings_profile, Length: 17880, dtype: object

time: 24.9 s (started: 2023-10-12 18:47:03 +00:00)


In [ ]:
# flatten the output
df['profile_embeddings'] = ''
for i in range(len(df)):
  embeddings = df['embeddings_profile'][i][0]
  new_embeddings = []
  for j in range(len(embeddings)):
    float_values = [float(value) for value in embeddings[j]]
    new_embeddings.append(float_values)
  df['profile_embeddings'].iloc[i] = new_embeddings

<ipython-input-41-516bfe0bd212>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['profile_embeddings'].iloc[i] = new_embeddings


In [ ]:
df['profile_embeddings']

0        [[-0.22073444724082947, -0.14457353949546814, ...
1        [[-0.38851073384284973, -0.1363488882780075, -...
2        [[-0.008704169653356075, 0.035887934267520905,...
3        [[-0.2483062595129013, 0.21164652705192566, 0....
4        [[-0.12102112174034119, -0.35673341155052185, ...
                               ...                        
17875                                                     
17876                                                     
17877                                                     
17878                                                     
17879                                                     
Name: profile_embeddings, Length: 17880, dtype: object

time: 436 ms (started: 2023-10-12 19:33:52 +00:00)


In [ ]:
df = pd.read_csv('/content/drive/My Drive/BT4012 Group 06/Data/feature_engineering_data_new.csv')
df.head()

,job_id,title,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,...,number of sentences_benefits,number of stopwords_profile_description,number of stopwords_profile,number of stopwords_requirements,number of stopwords_benefit,sentiment score_profile,sentiment score_requirement,sentiment score_benefits,sentiment score_description,embeddings_profile
0,1,Marketing Intern,Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,unknown,0,1,...,1,45,49,42,0,0.9856,0.7345,0.0000,0.6486,"tensor([[[-0.2207, -0.1446, -0.2817, ..., 0...."
1,2,Customer Service - Cloud Video Production,Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,...,5,104,43,68,112,0.9313,0.9929,0.9716,0.9951,"tensor([[[-0.3885, -0.1363, -0.0421, ..., -0...."
2,3,Commissioning Machinery Assistant (CMA),Sales,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,unknown,0,1,...,1,17,61,61,0,0.9618,0.9260,0.0000,0.9509,"tensor([[[-8.7042e-03, 3.5888e-02, -4.8977e-0..."
3,4,Account Executive - Washington DC,Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,...,1,122,29,55,22,0.9612,0.9819,0.8462,0.9957,"tensor([[[-0.2483, 0.2116, 0.1013, ..., -0...."
4,5,Bill Review Manager,Sales,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,...,1,44,53,26,0,0.9913,0.4019,0.3818,0.9426,"tensor([[[-0.1210, -0.3567, -0.1483, ..., 0...."


time: 6.52 s (started: 2023-10-18 01:34:21 +00:00)


In [ ]:
# Load the pre-trained Sentence-BERT model
model_name = "roberta-base-nli-mean-tokens"
model = SentenceTransformer(model_name)

df["noise_removed_company_profile"].iloc[0:1].apply(lambda x: model.encode(x, convert_to_tensor=True))

0    [tensor(0.0720), tensor(0.1265), tensor(0.0074...
Name: noise_removed_company_profile, dtype: object

time: 9.68 s (started: 2023-10-18 01:34:32 +00:00)


In [ ]:
cols_to_embed = ['title','company_profile','description','requirements','benefits']
len(df["noise_removed_company_profile"].iloc[0:1].apply(lambda x: model.encode(x, convert_to_tensor=True)).iloc[0])
sent_embedding_features = df[['job_id']]
for col in cols_to_embed:
  embed_result = df['noise_removed_' + col].apply(lambda x: model.encode(x, convert_to_tensor=True).numpy())
  embed_result = pd.DataFrame(np.array(embed_result.tolist())).rename(columns=lambda x: col + '_' + str(x))
  sent_embedding_features = pd.concat([sent_embedding_features, embed_result], axis=1)
  print(col + ' embedded')

title embedded
company_profile embedded
description embedded
requirements embedded
benefits embedded
time: 4h 35min 19s (started: 2023-10-18 01:34:49 +00:00)


In [ ]:
sent_embedding_features

job_id   title_0   title_1   title_2   title_3   title_4   title_5  \
0          1.0 -0.212008 -0.578243  0.439003  0.272899 -1.182581  1.799482   
1          2.0  0.059300  0.195787 -0.334536 -0.615924  0.073257  1.006594   
2          3.0  0.025818 -0.028397  0.641024 -0.271912 -1.435301  0.763417   
3          4.0 -0.034365 -0.072459  1.060798  0.112226 -1.297718  1.433325   
4          5.0  0.359326  0.113940  1.083886  0.947914 -1.008393  0.959356   
...        ...       ...       ...       ...       ...       ...       ...   
17875  17876.0  0.265625 -0.685699  0.824831  0.901035 -1.581751  1.147339   
17876  17877.0 -0.274574  0.154526  0.551615  0.270313 -0.779550  0.566122   
17877  17878.0 -0.358277  0.052849  0.176503 -0.407705 -2.090245  0.699344   
17878  17879.0  0.364278 -0.452159  0.205622 -0.376398 -0.485262  0.730838   
17879  17880.0  0.313351  0.061442  0.329944  0.199430 -1.712028  1.370470   

        title_6   title_7   title_8   title_9  title_10  title_11  title_12  \
0      0.660702 -0.966664 -1.062968  0.559611 -0.316995  1.479915  0.034776   
1      0.589629 -0.448304 -0.438520  0.164654  0.870841 -0.122048  0.457850   
2      0.380985  0.039243 -1.276695  0.573555  0.198223  1.121411 -0.461966   
3      0.730695  0.466384 -0.550613  1.109785 -0.197921  1.501082  0.166875   
4      0.548813  0.811003 -0.945536  0.906979  0.058713  0.025424  0.339909   
...         ...       ...       ...       ...       ...       ...       ...   
17875  0.705996  0.789471 -0.077398  1.082341 -0.065460  0.102903  0.453900   
17876  0.588223  0.284024 -0.525868  0.492443 -0.252622  0.173314  0.571647   
17877  0.323693 -0.160966 -1.460325  0.726354  0.095846 -0.969201  0.287222   
17878 -0.394595  0.268025 -0.078670 -0.193968  0.273451 -0.750284  0.232442   
17879  0.214716 -0.237277 -0.192295  0.179150  0.488132 -0.539384  0.501450   

       title_13  title_14  title_15  title_16  title_17  title_18  title_19  \
0     -0.232574  0.389990  0.519327 -0.361169 -0.719644 -0.613086 -0.679342   
1      0.023010  0.932175  0.265120 -1.049661 -0.510018  0.539764 -1.109684   
2     -0.320227  1.683018  1.436297 -0.272492 -1.027377 -0.528133 -1.224232   
3     -0.198842  0.807572  0.899447 -1.248963 -0.633742 -0.566714 -0.366122   
4      0.189164  0.211139  0.597166 -0.025881 -0.875369  0.710020 -0.760034   
...         ...       ...       ...       ...       ...       ...       ...   
17875  0.104714  0.246506  0.462184  0.113687 -1.150971 -0.738761  0.236140   
17876 -0.917156  0.835108  0.713931 -0.092060 -1.192400  0.782400 -0.802899   
17877  1.463509  1.251397  0.384390 -0.193413 -0.819692 -0.398945  0.191314   
17878 -0.471148  1.568790  0.199560 -0.130707  0.519741 -0.334337 -1.114785   
17879  1.202846  0.603574  0.176154  0.382951 -0.943752  0.156417 -0.309877   

       title_20  title_21  title_22  title_23  title_24  title_25  title_26  \
0     -0.043241 -0.003852 -0.421952  0.802283  0.798969 -0.291596 -0.431506   
1     -0.487101  0.560315  0.341604  0.449326 -0.424116 -0.584556  0.102889   
2     -0.372234  0.280675  0.949575 -0.233134  0.368549 -0.593184 -0.682265   
3      0.526906  0.570951  0.061095 -0.016558 -0.337801 -0.362297 -0.134999   
4     -0.160883 -0.120462  0.414093  0.446324  1.122974 -0.418372 -0.058035   
...         ...       ...       ...       ...       ...       ...       ...   
17875 -0.020770  0.118993  0.133518  0.131910  0.233600 -0.155173 -0.444815   
17876  0.035478  0.172381  0.920511  0.773332 -0.102869 -0.053233  0.542342   
17877  0.023125  0.214153  0.464791 -0.279314  0.308009 -0.193127 -0.201076   
17878 -0.147248  0.603895  0.156261  0.788304  0.770829 -0.613972  0.192023   
17879 -0.879936 -0.420043 -0.054334  0.162192  0.161675 -0.502045 -0.118437   

       title_27  title_28  title_29  title_30  title_31  title_32  title_33  \
0     -0.225456 -0.128093  0.700603 -0.108381  0.555930  0.269226 -0.272728   
1      0.019716 -0.576373 -0.168838  0.625596 -0.365299 -0.234656 

time: 2 s (started: 2023-10-18 07:50:38 +00:00)


In [ ]:
# Save feature dataframe to a new CSV file
sent_embedding_features.to_csv('sent_embedding_features.csv', index=False)

time: 2min 15s (started: 2023-10-18 06:10:09 +00:00)
